In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv("../data/processed/amazon_mi_clean.csv")
df.head(3)


,text,rating,summary,asin,reviewTime,label,text_clean
0,"According to my order history, I bought this t...",4.0,A bargain-bin good-enough ukulele that's held ...,0739079891,"04 10, 2016",1,ccording order history bought kit towards end ...
1,Please pay attention better than I did to the ...,1.0,Poor Quality product.,0739079891,"02 6, 2016",0,lease pay attention better reviews poor qualit...
2,I purchased this as a gift for a friend who re...,4.0,Good Starter Kit,0739079891,"01 2, 2016",1,purchased gift friend really wanted learn neat...


In [4]:
#label distribution
ax = df["label"].value_counts().sort_index().rename({0:"Neg",1:"Pos"}).plot(kind='bar')
ax.set_title("Label Distribution")
ax.set_ylabel("count")
fig = ax.get_figure()
Path("report/figures").mkdir(parents=True,exist_ok=True)
fig.savefig("report/figures/eda_label_distribution.png",dpi=150,bbox_inches="tight")
plt.close(fig)
df['label'].value_counts(normalize=True).round(3)


label
1    0.724
0    0.276
Name: proportion, dtype: float64

In [6]:
# text length
df['len']=df['text_clean'].str.split().map(len)
ax=df['len'].plot(kind='hist',bins=50)
ax.set_title('Review Length(tokens)')
ax.set_xlabel('tokens')
fig=ax.get_figure()
fig.savefig("report/figures/eda_length_hist.png",dpi=150,bbox_inches="tight")
plt.close(fig)
df['len'].describe()


count    53244.000000
mean        36.640861
std         56.678344
min          1.000000
25%          8.000000
50%         19.000000
75%         42.000000
max       1657.000000
Name: len, dtype: float64

In [7]:
# top tokens
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=10,max_df=0.9)
X =vect.fit_transform(df["text_clean"])
counts = X.sum(axis=0).A1
vocab=vect.get_feature_names_out()
top=pd.DataFrame({"token":vocab,'count':counts}).sort_values('count',ascending=False).head(30)
top

,token,count
3443,good,17637
5357,one,17116
4508,like,16228
3548,guitar,15769
7336,sound,14750
8500,use,13408
8910,would,13357
3487,great,12124
3377,get,11853
8762,well,11343


In [9]:
# save the top tokens
top.to_csv("report/figures/eda_top_tokens.csv",index=False)
top.plot(kind="bar",x="token",y="count",figsize=(10,4), legend=False)
plt.title('Top Tokens')
plt.tight_layout()
plt.savefig("report/figures/eda_top_tokens.png",dpi=150)
plt.close()
